In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [25]:
filename = "fake_news_dataset.csv"

# CSV файлды оқу
try:
    df = pd.read_csv(filename)
    print(f"{filename} файлы сәтті жүктелді!")
except FileNotFoundError:
    print(f"Қате: {filename} файлы табылмады. Файлды жүктегеніңізге көз жеткізіңіз.")

# Деректерге шолу
print(f"Деректер өлшемі: {df.shape}")
print(df.head())

# Керек емес бағандарды алып тастау (бізге тек 'text' және 'label' керек)
# Сіздің файлда баған аты 'text' және 'label' деп тұр (кіші әріппен)
df = df[['text', 'label']]

# Бос мәндерді (NaN) тексеру және жою
df.dropna(inplace=True)

# 'label' бағанын сандық мәнге айналдыру (Fake -> 0, Real -> 1)
# LabelEncoder автоматты түрде кластарды 0 және 1-ге айналдырады
le = LabelEncoder()
df['label_num'] = le.fit_transform(df['label'])

print("\nКластардың кодталуы (Mapping):")
for i, label in enumerate(le.classes_):
    print(f"{label} -> {i}")

print("\nКластардың таралуы:")
print(df['label'].value_counts())

fake_news_dataset.csv файлы сәтті жүктелді!
Деректер өлшемі: (4000, 24)
   id            title        author  \
0   1  Breaking News 1    Jane Smith   
1   2  Breaking News 2   Emily Davis   
2   3  Breaking News 3      John Doe   
3   4  Breaking News 4  Alex Johnson   
4   5  Breaking News 5   Emily Davis   

                                                text           state  \
0  This is the content of article 1. It contains ...       Tennessee   
1  This is the content of article 2. It contains ...       Wisconsin   
2  This is the content of article 3. It contains ...        Missouri   
3  This is the content of article 4. It contains ...  North Carolina   
4  This is the content of article 5. It contains ...      California   

  date_published          source       category  sentiment_score  word_count  \
0     30-11-2021       The Onion  Entertainment            -0.22        1302   
1     02-09-2021    The Guardian     Technology             0.92         322   
2     13-04-20

In [28]:
X = df['text']
y = df['label_num']

tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
X_vec = tfidf.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)

In [27]:
models = {
    "KNN (k=5)": KNeighborsClassifier(n_neighbors=5),
    "SVM (Linear)": SVC(kernel='linear', random_state=42),
    "Decision Tree": DecisionTreeClassifier(max_depth=10, random_state=42)
}

results = {}

for name, model in models.items():
    # Модельді үйрету
    model.fit(X_train, y_train)

    # Болжау
    y_pred = model.predict(X_test)

    # Нәтижелерді есептеу
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    results[name] = (acc, cm)
    print(f"{name:20} | Accuracy: {acc:.4f}")

KNN (k=5)            | Accuracy: 0.5138
SVM (Linear)         | Accuracy: 0.5138
Decision Tree        | Accuracy: 0.5138


In [29]:
best_model_name = max(results, key=lambda k: results[k][0])
best_model_name

'KNN (k=5)'